# CARTO Data Observatory

This is a basic template notebook to start exploring your new Dataset from CARTO's Data Observatory via the Python library [CARTOframes](https://carto.com/cartoframes).

You can find more details about how to use CARTOframes in the [Quickstart guide](https://carto.com/developers/cartoframes/guides/Quickstart/).

# Setup

## Installation

Make sure that you have the latest version installed. Please, find more information in the [Installation guide](https://carto.com/developers/cartoframes/guides/Installation/).

In [ ]:
!pip install -U cartoframes

In [ ]:
# Note: a kernel restart is required after installing the library

import cartoframes

cartoframes.__version__

## Credentials

In order to be able to use the Data Observatory via CARTOframes, you need to set your CARTO account credentials first.

In [ ]:
from cartoframes.auth import set_default_credentials

username = 'YOUR_USERNAME'
api_key  = 'YOUR_API_KEY'

set_default_credentials(username, api_key)

**Note about credentials**

For security reasons, we recommend storing your credentials in an external file preventing publishing them by accident within your notebook. You can get more information in the section *Setting your credentials* of the [Authentication guide](https://carto.com/developers/cartoframes/guides/Authentication/).

# Dataset operations

## Metadata exploration

In this section we will explore some basic information regarding the Dataset you have licensed. More information on how to explore metadata associated to a Dataset is available in the [Data discovery guide](https://carto.com/developers/cartoframes/guides/Data-discovery/).

In order to access the Dataset and its associeted metadata, you need to provide the "Slug ID" which is a unique identifier of that Dataset. The Slug IDs of your Datasets are available from Your Subscriptions page in the CARTO Dashboard and via the Discovery methods in CARTOFrames.

In [ ]:
from cartoframes.data.observatory import Dataset

dataset = Dataset.get('YOUR_SLUG_ID')

In [ ]:
# Retrieve some general metadata about the Dataset
dataset.to_dict()

In [ ]:
# Explore the first 10 rows of the Dataset
dataset.head()

In [ ]:
# Explore the last 10 rows of the Dataset
dataset.tail()

In [ ]:
# Get the geographical coverage of the data
dataset.geom_coverage()

In [ ]:
# Access the list of variables in the dataset
dataset.variables.to_dataframe()

In [ ]:
# Summary of some variable stats
dataset.describe()

## Access the data

Now that we have explored some basic information about the Dataset, we will proceed to download a sample of the Dataset into a dataframe so we can operate it in Python. 

Datasets can be downloaded in full or by applying a filter with a SQL query. More info on how to download the Dataset or portions of it is available in the [Data discovery guide](https://carto.com/developers/cartoframes/guides/Data-discovery/).

In [ ]:
# Filter by SQL query
query = "SELECT * FROM $dataset$ LIMIT 100"

dataset_df = dataset.to_dataframe(sql_query=query)

**Note about SQL filters**

Our SQL filtering queries allow for any PostgreSQL and PostGIS operation, so you can filter the rows (by a WHERE condition) or the columns (using the SELECT). Some common examples are filtering the Dataset by bounding box or filtering by column value: 

```
SELECT * FROM $dataset$ WHERE ST_IntersectsBox(geom, -74.044467,40.706128,-73.891345,40.837690)
```

```
SELECT total_pop, geom FROM $dataset$
```

A good tool to get the bounding box details for a specific area is [bboxfinder.com](http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000).

In [ ]:
# First 10 rows of the Dataset sample
dataset_df.head()

## Visualization

Now that we have downloaded some data into a dataframe we can leverage the visualization capabilities of CARTOframes to build an interactive map.

More info about building visualizations with CARTOframes is available in the [Visualization guide](https://carto.com/developers/cartoframes/guides/Visualization/).

In [ ]:
from cartoframes.viz import Layer

Layer(dataset_df, geom_col='geom')

**Note about variables**

CARTOframes allows you to make data-driven visualizations from your Dataset variables (columns) via the Style helpers. These functions provide out-of-the-box cartographic styles, legends, popups and widgets.

Style helpers are also highly customizable to reach your desired visualization setting simple parameters. The helpers collection contains functions to visualize by color and size, and also by type: category, bins and continuous, depending on the type of the variable.

In [ ]:
from cartoframes.viz import color_bins_style

Layer(dataset_df, color_bins_style('YOUR_VARIABLE'),geom_col='geom')

## Upload to CARTO account

In order to operate with the data in CARTO Builder or to build a CARTOFrames visualization reading the data from a table in the Cloud instead of having it in your local environment (with its benefits in performance), you can load the dataframe as a table in your CARTO account.

More info in the [Data Management guide](https://carto.com/developers/cartoframes/guides/Data-management/).

In [ ]:
from cartoframes import to_carto

to_carto(dataset_df, 'my_dataset', geom_col='geom')

In [ ]:
# Build a visualization reading the data from your CARTO account
Layer('my_dataset')

## Enrichment

Enrichment is the process of adding variables to a geometry, which we call the target (point, line, polygon…) from a spatial Dataset, which we call the source. CARTOFrames has a set of methods for you to augment your data with new variables from a Dataset in the Data Observatory.

In this example, you will need to load a dataframe with the geometries that you want to enrich with a variable or a group of variables from the Dataset. You can detail the variables to get from the Dataset by passing the variable's Slug ID. You can get the variables' Slug IDs with the metadata methods.

More info in the [Data enrichment guide](https://carto.com/developers/cartoframes/guides/Data-enrichment/).

In [ ]:
from cartoframes.data.observatory import Enrichment

enriched_df = Enrichment().enrich_polygons(
    df,  # Insert here the DataFrame to be enriched
    variables=['YOUR_DATASET_VARIABLE_SLUG_ID']
)

## Save to file

Finally, you can also export the data into a CSV file. More info in the [Data discovery guide](https://carto.com/developers/cartoframes/guides/Data-discovery/).

In [ ]:
# Filter by SQL query
query = "SELECT * FROM $dataset$ LIMIT 100"

dataset.to_csv('my_dataset.csv', sql_query=query)